# This colab notebook corresponds to Day 2 PyAEZ Training Agenda
## Part 1: Module I: Climate Regime
### Date: 30.11.2022
### Prepared by Geoinformatics Center, AIT

# Google drive connection



In [ ]:
# Connecting to my google drive
from google.colab import drive
drive.mount('drive/')

---
# Installing necessary packages

In [ ]:
!pip install gdal
!pip install pyaez==2.0.0

In [ ]:
# importing libraries
import matplotlib.pyplot as plt
import numpy as np
import os
from osgeo import gdal
import scipy

# Folder path to dataset folder

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/PyAEZ_Training_Materials_2022/Training_Materials/')

# Importing the datasets (For some datasets, we will use GDAL to convert to numpy array

In [ ]:
# importing the climate data
max_temp = np.load(r'./climate/max_temp.npy') # maximum temperature
min_temp = np.load(r'./climate/min_temp.npy') # minimum temperature
precip = np.load(r'./climate/precipitation.npy') # precipitation
rel_humid = np.load(r'./climate/relative_humidity.npy') # relative humidity
wind_sp = np.load(r'./climate/wind_speed.npy') # wind speed measured at two meters
short_rad = np.load(r'./climate/short_rad.npy') # shortwave radiation

mask = gdal.Open(r'./LAO_Admin.tif').ReadAsArray()
elevation = gdal.Open(r'./LAO_Elevation.tif').ReadAsArray()
soil_terrain_lulc = gdal_Open(r'./LAO_soil_terrain_lulc.tif').ReadAsArray()

# Module I:Climate Regime Start

In [ ]:
# importing Module I
from pyaez import ClimateRegime
clim_reg = ClimateRegime.ClimateRegime()

In [ ]:
# importing UtilitiesCalc
from pyaez import UtilitiesCalc
obj_util = UtilitiesCalc.UtilitiesCalc()

## Importing the data into the object class

In [ ]:
# Retrieve your target area's minimum and maximum latitude
lat_max = 22.59
lat_min = 13.87

In [ ]:
# setting the study area mask ***Optional
clim_reg.setStudyAreaMask(mask, 0)

# adding the location and elevation data ***Mandatory Step
clim_reg.setLocationTerrainData(lat_min, lat_max, elevation)

# setting up the climate data ***Mandatory Step
# Monthly Data
# clim_reg.setMonthlyClimateData(min_temp, max_temp, precip, short_rad, wind_sp, rel_humid)

# Daily Data ***Mandatory Step
clim_reg.setDailyClimateData(min_temp, max_temp, precip, short_rad, wind_sp, rel_humid, mask, elevations)

In [ ]:
# Releasing the memory of input climate data
del(min_temp, max_temp, precip, short_rad, wind_sp, rel_humid)

# Thermal Climate

In [ ]:
tclimate = clim_reg.getThermalClimate()

# now visualizing result
fig = plt.figure(figsize=(14, 10))
plt.imshow(tclimate, cmap = plt.get_cmap('tab20b', 12))
plt.title('Thermal Climate')
plt.colorbar()
plt.show()

In [ ]:
# saving the output as TIFF file
obj_util.saveRaster(r'./LAO_Elevation.tif', r'./tclimate.tif', tclimate)

# Thermal Zone

In [ ]:
tzone = clim_reg.getThermalZone()

# now visualizing result
fig = plt.figure(figsize=(14, 10))
plt.imshow(tclimate, cmap = plt.get_cmap('tab20b', 12))
plt.title('Thermal Zone')
plt.colorbar()
plt.show()

In [ ]:
# saving the output as TIFF file
obj_util.saveRaster(r'./LAO_Elevation.tif', r'./tzone.tif', tzone)

# Thermal Length of Growing Period

In [ ]:
lgp0 = clim_reg.getThermalLGP0() # 0 deg Celsius threshold
lgp5 = clim_reg.getThermalLGP5() # 5 deg Celsius threshold
lgp10 = clim_reg.getThermalLGP10() # 10 deg Celsius threshold

fig, ax = plt.subplots(nrows = 1, ncols = 3)
ax[0].imshow(lgp0, vmax = 366, vmin = 0)
ax[1].imshow(lgp5, vmax = 366, vmin = 0)
ax[2].imshow(lgp10, vmax = 366, vmin = 0)

ax[0].title.set_text('LGP0')
ax[1].title.set_text('LGP5')
ax[2].title.set_text('LGP10')

fig.colorbar(lgp10)
plt.show()

In [ ]:
# saving the output as TIFF file
obj_util.saveRaster(r'./LAO_Elevation.tif', r'./lgpt0.tif', lgp0)
obj_util.saveRaster(r'./LAO_Elevation.tif', r'./lgpt5.tif', lgp5)
obj_util.saveRaster(r'./LAO_Elevation.tif', r'./lgpt10.tif', lgp10)

# Temperature Summation

In [ ]:
tsum = clim_reg.getTemperatureSum0()
tsum5 = clim_reg.getTemperatureSum5()
tsum10 = clim_reg.getTemperatureSum10()

fig, ax = plt.subplots(nrows = 1, ncols = 3)
ax[0].imshow(lgp0, vmax = 18000, vmin = 0)
ax[1].imshow(lgp5, vmax = 18000, vmin = 0)
ax[2].imshow(lgp10, vmax = 18000, vmin = 0)

ax[0].title.set_text('Tsum0')
ax[1].title.set_text('Tsum5')
ax[2].title.set_text('LGP10')

fig.colorbar(lgp10)
plt.show()

# Temperature Profile

In [ ]:
tprofile = clim_reg.getTemperatureProfile()
tile_list = ['A9','A8','A7','A6','A5','A4','A3','A2','A1','B1','B2','B3','B4','B5','B6','B7','B8','B9']

fig = plt.figure(figsize=(20,40))  
for i1 in range(1,19):
    plt.subplot(6, 3, i1)
    plt.imshow(tprofile[i1-1])
    plt.title(tile_list[i1-1])
    plt.colorbar() 
plt.tight_layout()
plt.show()

# Length of Growing Period/ Equivalent Length of Growing Period/ Moisture Regime

In [ ]:
# LGP
lgp = clim_reg.getLGP(Sa = 100)

# Equivalent LGP
lgp_equv = clim_reg.getLGPEquivalent()

fig, ax = plt.subplots(nrows = 1, ncols = 2)
ax[0].imshow(lgp, vmax = 366, vmin = 0)
ax[1].imshow(lgp10, vmax = 366, vmin = 0)

ax[0].title.set_text('Length of Growing Period')
ax[1].title.set_text('Equivalent Length of Growing Period')

plt.show()

In [ ]:
# saving the output as TIFF file
obj_util.saveRaster(r'./LAO_Elevation.tif', r'./LGP.tif', lgp)
obj_util.saveRaster(r'./LAO_Elevation.tif', r'./lgp_equv.tif', lgp_equv)

# Multi-cropping Zonation

In [ ]:
multi_crop = clim_reg.getMultiCroppingZones(tclimate, lgp, lgpt5, lgpt10, tsum10, tsum0)

multi_crop_rainfed = multi_crop[0] # for rainfed conditions
multi_crop_irr = multi_crop[1] # for irrigated conditions

fig, ax = plt.plots(nrows = 1, ncols = 2)
ax[0].imshow(multi_crop_rainfed, vmax = 8, vmin = 0)
ax[1].imshow(multi_crop_irr, vmax = 8, vmin = 0)

ax[0].title.set_text('Multi-cropping Zone (Rainfed)')
ax[1].title.set_text('Multi-cropping Zone (Irrigated)')

plt.show()

# Frost Index and Permafrost Evaluation

In [ ]:
fi = clim_reg.AirFrostIndexandPermafrostEvaluation()

afi = fi[0] # Air Frost Index
permafrost = fi[1] # Classified Permafrost Classes

fig, ax = plt.plots(nrows = 1, ncols = 2)
ax[0].imshow(afi, vmax = 1, vmin = 0)
ax[1].imshow(permafrost, vmax = 4, vmin = 0)

ax[0].title.set_text('Air Frost Index')
ax[1].title.set_text('Permafrost Classes')

plt.show()

In [ ]:
# saving the output as TIFF file
obj_util.saveRaster(r'./LAO_Elevation.tif', r'./airfrostindex.tif', afi)
obj_util.saveRaster(r'./LAO_Elevation.tif', r'./permafrost_class.tif', permafrost)

# Thermal Zone for Fallow Requirements

In [ ]:
tzone_fallow = clim_reg.TZoneFallowRequirement(tzone)

# now visualizing result
fig = plt.figure(figsize=(14, 10))
plt.imshow(tzone_fallow, cmap = plt.get_cmap('Accent', 7))
plt.title('Thermal Zone for Fallow Requirement')
plt.colorbar()
plt.show()

In [ ]:
# saving the output as TIFF file
obj_util.saveRaster(r'./LAO_Elevation.tif', r'./Tzonefallow.tif', tzone_fallow)

# Agro-ecological Zone Classification

In [ ]:
aez = clim_reg.AEZClassification(tclimate, lgp, lgp_equv, lgpt5, soil_terrain_lulc, permafrost)

# now visualizing result
fig = plt.figure(figsize=(14, 10))
plt.imshow(aez, cmap = plt.get_cmap('Rainbow', 58))
plt.title('Agro-ecological Zonation')
plt.colorbar()
plt.show()

# End of Module I: Climate Regime
---


# Module II: Crop Simulation Start

In [ ]:
# importing Module I
from pyaez import CropSimulation
aez = CropSimulation.CropSimulation()

In [ ]:
# importing the climate data
max_temp = np.load(r'./climate/max_temp.npy') # maximum temperature
min_temp = np.load(r'./climate/min_temp.npy') # minimum temperature
precip = np.load(r'./climate/precipitation.npy') # precipitation
rel_humid = np.load(r'./climate/relative_humidity.npy') # relative humidity
wind_sp = np.load(r'./climate/wind_speed.npy') # wind speed measured at two meters
short_rad = np.load(r'./climate/short_rad.npy') # shortwave radiation

mask = gdal.Open(r'./LAO_Admin.tif').ReadAsArray()
elevation = gdal.Open(r'./LAO_Elevation.tif').ReadAsArray()

# Importing the data into the object class (Setting up climate data, geographic location data and mask layer) (Mandatory)

In [ ]:
# Reading the climate data into Crop Simulation Object Class ***Mandatory step
# For monthly data 
aez.setMonthlyClimateData(min_temp, max_temp, precip, short_rad, wind_sp, rel_humid) 

# For daily data
# aez.setDailyClimateData(min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity)

# Setting up the location and elevation  ***Mandatory step
aez.setLocationTerrainData(lat_min= lat_min, lat_max = lat_max , elevation = elevation)

# Setting the study area mask
aez.setStudyAreaMask(admin_mask = mask, no_data_value = 0)

# Setting up the crop parameter/ crop cycle parameter and soil water parameters (Mandatory)

In [ ]:
# setting up the crop parameters, crop cycle and soil water parameters ***mandatory step
aez.setCropParameters(LAI=2.3, HI=0.33, legume=0, adaptability=4, cycle_len=115, D1=0.3, D2=1)

aez.setCropCycleParameters(stage_per=[16, 26, 33, 25], kc=[0.3, 1.2, 0.5], kc_all=0.85, yloss_f=[0.4, 0.4, 0.9, 0.5], yloss_f_all=1.25)

aez.setSoilWaterParameters(Sa= 100*np.ones((mask.shape)), pc=0.5)

# Setting up the thermal screening parameters (Optional)

In [ ]:
# If you're simulating perennial crops, this thermal screening is mandatory
aez.adjustForPerennialCrop(aLAI =70, bLAI = 200, aHI = 120, bHI = 180)

# Thermal Climate screening
aez.setThermalClimateScreening(tclimate, no_t_tclimate = [2,5,6])

# Temperature Growing Period Screening (Need to clarify)
aez.setLGPTScreening(no_lgpt = [2100 ,3600], optm_lgpt = [2300, 3150])

# Temperature Summation Screening (Need to clarify)
aez.setTSumScreening(no_Tsum = [2100 ,3600], optm_Tsum = [2300, 3150])

# Temperature Profile Screening (Need to clarify)
aez.setTProfileScreening(no_Tprofile = [])

# Simulation of Crop Cycle

In [ ]:
aez.simulateCropCycle(start_doy = 1, end_doy = 365, step_doy = 1, leap_year = False)

# Maximum Attainable Yield for Rainfed and Irrigated Condition/ Optimum starting date


Yield Classification
1.   not suitable (yields between 0% and 20%)
2.   marginally suitable (yields between 20% and 40%)
3.   moderately suitable (yields between 40% and 60%)
4. suitable (yields between 60% and 80%)
5. very suitable (yields are equivalent to 80% or more of the overall maximum yield)



In [ ]:
# Now, showing the estimated and highly obtainable yield of a particular crop, results are in kg / hectare
yield_map_rain = aez.getEstimatedYieldRainfed() # for rainfed
yield_map_irr = aez.getEstimatedYieldIrrigated() # for irrigated

starting_date = aez.getOptimumCycleStartDate() # Optimum cycle start date, the date when the highest yield are produced referenced from the start of crop cycle

## get classified output of yield 
yield_map_rain_class = obj_util.classifyFinalYield(yield_map_rain)
yield_map_irr_class = obj_util.classifyFinalYield(yield_map_irr)

In [ ]:
# Visualization of result
fig, ax = plt.subplots(nrows = 3, ncols = 2)
fig.delaxes(ax[2,1])

maxi_value = np.max([yield_map_rain, yield_map_irr])

ax[0,0].imshow(yield_map_rain, vmax = maxi_value, vmin = 0)
ax[0,1].imshow(yield_map_rain_class, vmax = 366, vmin = 0)

ax[1,0].imshow(yield_map_irr, vmax = maxi_value, vmin = 0)
ax[1,1].imshow(yield_map_irr_class, vmax = 366, vmin = 0)

ax[2,0].imshow(starting_date, vmax = 366, vmin = 0)


ax[1,0].title.set_text('yield_map_rain')
ax[1,1].title.set_text('yield_map_rain_class')
ax[1,1].title.set_text('yield_map_irr')
ax[1,1].title.set_text('yield_map_irr_class')
ax[1,1].title.set_text('Optimum starting_date')

plt.show()

In [ ]:
# saving the output as TIFF file
obj_util.saveRaster(r'./LAO_Elevation.tif', r'./yield_map_rain.tif', yield_map_rain)
obj_util.saveRaster(r'./LAO_Elevation.tif', r'./yield_map_rain_class.tif', yield_map_rain_class)

obj_util.saveRaster(r'./LAO_Elevation.tif', r'./yield_map_irr.tif', yield_map_irr)
obj_util.saveRaster(r'./LAO_Elevation.tif', r'./yield_map_irr_class.tif', yield_map_irr_class)

obj_util.saveRaster(r'./LAO_Elevation.tif', r'./starting_date.tif', starting_date)

---
# End of Module II: Crop Simulation